In [1]:
import pandas as pd
import numpy as np

In [2]:
#import csv
data_train = pd.read_csv('data_train.csv')

      battery_power  blue  clock_speed  dual_sim  fc  four_g  int_memory  \
0               804     1          0.8         1  12       1          41   
1              1042     0          2.2         0  15       1          11   
2              1481     1          2.0         1   0       0          35   
3              1104     0          1.7         0   1       1          60   
4               652     0          0.5         1   1       0          58   
...             ...   ...          ...       ...  ..     ...         ...   
1395            536     1          1.4         0   0       1          53   
1396           1097     0          0.8         0  10       1          21   
1397           1179     1          0.5         0   7       1          32   
1398            719     1          0.5         1   0       1          23   
1399           1439     0          0.9         0  12       1          20   

      m_dep  mobile_wt  n_cores  ...  px_height  px_width   ram  sc_h  sc_w  \
0       

In [3]:
#Memilih kolom yang paling berpengaruh pada penilaian
data_train = data_train[["battery_power", "mobile_wt", "px_height", "px_width", "ram", "price_range"]]
data_train

,battery_power,mobile_wt,px_height,px_width,ram,price_range
0,804,89,709,818,2027,1
1,1042,139,68,1018,2826,2
2,1481,105,249,522,2635,2
3,1104,199,653,1413,1229,0
4,652,142,464,781,565,0
...,...,...,...,...,...,...
1395,536,135,547,705,1211,0
1396,1097,160,1277,1352,2219,2
1397,1179,182,85,1451,340,0
1398,719,113,431,1727,3990,3


In [4]:
#Mengubah kolom dari nilai numerik menjadi nilai kategorikal
def categorical(dataset):
    batas = [[864.75, 1219.9, 1655.75], [111.0, 143.0, 172.0], [307.0, 576.5, 928.5], [874.0, 1249.0, 1639.75], [1220.75, 2286.0, 3114.5]]
    nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
    for i in range(len(batas)):
        dataset.loc[dataset[nama[i]] <= batas[i][0], nama[i]] = 0
        dataset.loc[(dataset[nama[i]] > batas[i][0]) & (dataset[nama[i]] <= batas[i][1]), nama[i]] = 1
        dataset.loc[(dataset[nama[i]] > batas[i][1]) & (dataset[nama[i]] <= batas[i][2]), nama[i]] = 2
        dataset.loc[dataset[nama[i]] > batas[i][2], nama[i]] = 3
    return dataset

categorical_data_train = categorical(data_train)
categorical_data_train

,battery_power,mobile_wt,px_height,px_width,ram,price_range
0,0,0,2,0,1,1
1,1,1,0,1,2,2
2,2,0,0,0,2,2
3,1,3,2,2,1,0
4,0,1,1,0,0,0
...,...,...,...,...,...,...
1395,0,1,1,0,0,0
1396,1,2,3,2,1,2
1397,1,3,0,2,0,0
1398,0,1,1,3,3,3


In [5]:
# kalkulasi P(Vj)
def p_vj(data_train):
    list = []
    for i in np.unique(data_train["price_range"]):
        count = sum(data_train["price_range"] == i)
        list.append(count/len(data_train))
    return list

prior = p_vj(categorical_data_train)
print(prior)

[0.2557142857142857, 0.2542857142857143, 0.24642857142857144, 0.24357142857142858]


In [6]:
# kalkulasi P(Ai|Vj)
def p_ai_vj(data_train):
    nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
    list = []
    for i in range(len(nama)):
        kolom = []
        print(i)
        for j in np.unique(data_train["price_range"]):
            print("price range ", j)
            matrix = []
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 0))
            matrix.append(count/sum(data_train["price_range"] == j))
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 1))
            matrix.append(count/sum(data_train["price_range"] == j))
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 2))
            matrix.append(count/sum(data_train["price_range"] == j))
            count = sum((data_train["price_range"] == j) & (data_train[nama[i]] == 3))
            matrix.append(count/sum(data_train["price_range"] == j))
            kolom.append(matrix)
        list.append(kolom)    
    return list

likelihood = p_ai_vj(categorical_data_train)

# print matrix likelihood
for i in range(len(likelihood)):
    print("matrix likelihood ", i)
    for j in range(len(likelihood[i])):
        print(likelihood[i][j])

0
price range  0
price range  1
price range  2
price range  3
1
price range  0
price range  1
price range  2
price range  3
2
price range  0
price range  1
price range  2
price range  3
3
price range  0
price range  1
price range  2
price range  3
4
price range  0
price range  1
price range  2
price range  3
matrix likelihood  0
[0.31564245810055863, 0.32122905027932963, 0.2346368715083799, 0.12849162011173185]
[0.23876404494382023, 0.26685393258426965, 0.27247191011235955, 0.22191011235955055]
[0.2782608695652174, 0.22028985507246376, 0.2811594202898551, 0.22028985507246376]
[0.16422287390029325, 0.1906158357771261, 0.3343108504398827, 0.31085043988269795]
matrix likelihood  1
[0.2709497206703911, 0.22905027932960895, 0.23743016759776536, 0.26256983240223464]
[0.26685393258426965, 0.23876404494382023, 0.25, 0.2443820224719101]
[0.24347826086956523, 0.2579710144927536, 0.27246376811594203, 0.22608695652173913]
[0.34310850439882695, 0.25806451612903225, 0.22287390029325513, 0.1759530791

In [7]:
# convert likelihood and prior to model on txt
def convert_to_txt(prior, likelihood):
    file = open("model.txt", "w")
    file.write("Prior\n")
    for i in range(len(prior)):
        file.write(str(prior[i]))
        file.write("\n")
    file.write("Likelihood\n")
    for i in range(len(likelihood)):
        for j in range(len(likelihood[i])):
            file.write(str(likelihood[i][j]))
            file.write("\n")
    file.close()

convert_to_txt(prior, likelihood)

Validasi Data Latih

In [8]:
import numpy as np
import pandas as pd

In [9]:
data_validation = pd.read_csv('data_validation.csv')
data_validation = data_validation[["battery_power", "mobile_wt", "px_height", "px_width", "ram", "price_range"]]
categorical_data_validation = categorical(data_validation)
categorical_data_validation

,battery_power,mobile_wt,px_height,px_width,ram,price_range
0,0,3,0,0,2,1
1,0,2,3,3,2,2
2,0,3,0,0,3,3
3,0,2,2,3,0,0
4,0,1,1,0,3,3
...,...,...,...,...,...,...
595,2,1,1,1,1,1
596,3,1,0,0,3,3
597,0,3,1,3,0,0
598,3,3,2,3,1,2


In [10]:
# # menghitung posterior

def posterior(data_validation, prior, likelihood):
    nama = ["battery_power", "mobile_wt", "px_height", "px_width", "ram"]
    list = []
    for i in range(len(data_validation)):
        matrix = []
        for j in range(len(prior)):
            matrix.append(prior[j])
            for k in range(len(nama)):
                matrix[j] = matrix[j] * likelihood[k][j][data_validation[nama[k]][i]]

        list.append(matrix)
    return list

Posterior = posterior(categorical_data_validation, prior, likelihood)


In [11]:
def predict(Posterior, data_validation):
    list = []
    for i in range(len(Posterior)):
        list.append(np.argmax(Posterior[i]))
    count = 0
    for i in range(len(data_validation)):
        if list[i] == data_validation["price_range"][i]:
            count += 1

    return list, count/len(data_validation)

hasil, a = predict(Posterior, categorical_data_validation)
print(hasil)
print(a)

[2, 2, 3, 0, 3, 1, 3, 0, 3, 1, 2, 2, 3, 0, 3, 0, 2, 1, 0, 2, 2, 1, 0, 1, 1, 1, 2, 2, 0, 2, 1, 3, 3, 1, 2, 3, 1, 3, 1, 1, 0, 2, 0, 2, 2, 1, 1, 3, 1, 3, 1, 2, 3, 0, 1, 3, 2, 3, 3, 1, 1, 3, 3, 0, 3, 2, 2, 2, 2, 2, 2, 3, 1, 0, 1, 2, 0, 2, 1, 0, 3, 3, 0, 2, 3, 0, 3, 3, 0, 3, 1, 1, 0, 2, 1, 0, 3, 2, 1, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 2, 3, 1, 0, 3, 1, 3, 0, 2, 2, 3, 2, 1, 2, 2, 1, 3, 3, 1, 0, 0, 3, 0, 0, 1, 3, 0, 1, 3, 3, 1, 2, 2, 1, 2, 1, 2, 3, 1, 0, 2, 1, 0, 3, 0, 0, 3, 3, 3, 0, 2, 3, 0, 0, 0, 0, 2, 3, 1, 0, 1, 1, 1, 3, 0, 2, 0, 3, 1, 1, 2, 0, 2, 0, 3, 1, 0, 3, 2, 0, 1, 2, 0, 1, 3, 3, 1, 1, 2, 2, 3, 2, 3, 3, 3, 0, 3, 0, 2, 3, 1, 2, 3, 2, 0, 0, 2, 2, 1, 3, 3, 0, 2, 0, 3, 0, 2, 1, 2, 2, 1, 3, 0, 1, 3, 3, 3, 0, 0, 1, 1, 3, 3, 1, 2, 0, 2, 1, 2, 3, 3, 0, 2, 1, 1, 1, 2, 0, 2, 2, 3, 3, 0, 1, 3, 3, 0, 1, 2, 1, 1, 1, 0, 3, 3, 2, 2, 0, 1, 0, 1, 1, 1, 1, 0, 3, 0, 3, 0, 1, 0, 1, 3, 2, 2, 0, 2, 1, 0, 1, 1, 1, 2, 0, 3, 1, 1, 0, 2, 3, 1, 3, 1, 3, 1, 2, 2, 0, 0, 2, 0, 2, 1, 0, 1, 1, 0, 3, 2, 3, 